In [1]:
%pip install mediapipe opencv-python

In [2]:
import cv2
import mediapipe as mp 
import numpy as np 
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

In [3]:
help(mp_drawing.DrawingSpec)

Help on class DrawingSpec in module mediapipe.python.solutions.drawing_utils:

class DrawingSpec(builtins.object)
 |  DrawingSpec(color: Tuple[int, int, int] = (224, 224, 224), thickness: int = 2, circle_radius: int = 2) -> None
 |  
 |  DrawingSpec(color: Tuple[int, int, int] = (224, 224, 224), thickness: int = 2, circle_radius: int = 2)
 |  
 |  Methods defined here:
 |  
 |  __eq__(self, other)
 |      Return self==value.
 |  
 |  __init__(self, color: Tuple[int, int, int] = (224, 224, 224), thickness: int = 2, circle_radius: int = 2) -> None
 |      Initialize self.  See help(type(self)) for accurate signature.
 |  
 |  __repr__(self)
 |      Return repr(self).
 |  
 |  ----------------------------------------------------------------------
 |  Data descriptors defined here:
 |  
 |  __dict__
 |      dictionary for instance variables (if defined)
 |  
 |  __weakref__
 |      list of weak references to the object (if defined)
 |  
 |  -------------------------------------------------

In [5]:
from IPython.display import Image

Image(url="https://i.imgur.com/3j8BPdc.png", width=300)

In [6]:
cap = cv2.VideoCapture(0)
## Setup mediapipe instance
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()
        
        # Recolor image to RGB
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
      
        # Make detection
        results = pose.process(image)
    
        # Recolor back to BGR
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        # Extract landmarks
        try:
            landmarks = results.pose_landmarks.landmark
            print(landmarks)
        except:
            pass
        
        # Render detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2), 
                                mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2) 
                                 )               
        
        cv2.imshow('Mediapipe Feed', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

[x: 0.5406701
y: 0.4528329
z: -1.4210922
visibility: 0.99988806
, x: 0.5798472
y: 0.38639095
z: -1.3409408
visibility: 0.99978536
, x: 0.6015987
y: 0.3906008
z: -1.340998
visibility: 0.9998105
, x: 0.6238401
y: 0.39579964
z: -1.3412398
visibility: 0.9997428
, x: 0.5119754
y: 0.37778977
z: -1.3453293
visibility: 0.99976665
, x: 0.48775142
y: 0.3769357
z: -1.3444388
visibility: 0.99979955
, x: 0.4646085
y: 0.3776778
z: -1.34485
visibility: 0.99974614
, x: 0.6657256
y: 0.44240788
z: -0.79481477
visibility: 0.9997929
, x: 0.42612737
y: 0.4116379
z: -0.79392195
visibility: 0.9998622
, x: 0.5819719
y: 0.5440599
z: -1.2156495
visibility: 0.9999156
, x: 0.49607173
y: 0.5380757
z: -1.216153
visibility: 0.99992776
, x: 0.81950676
y: 0.8711515
z: -0.4675185
visibility: 0.9986972
, x: 0.24135518
y: 0.7988186
z: -0.52784956
visibility: 0.9985145
, x: 0.95733243
y: 1.323357
z: -0.58109796
visibility: 0.19452074
, x: 0.041249007
y: 1.0478878
z: -0.75282997
visibility: 0.34192133
, x: 0.95714664
y: 1.

In [10]:
len(landmarks)

33

In [11]:
for lndmrk in mp_pose.PoseLandmark: 
    print(lndmrk)
    

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32


In [12]:
[landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y]

[0.3199075162410736, 0.12152671068906784]

In [13]:
[landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].y]

[0.23667044937610626, 0.4232039153575897]

In [14]:
[landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].y]

[0.2163674384355545, 0.6843536496162415]

In [17]:
def calculate_overhead_press_angle(right_shoulder, right_elbow, right_wrist):
    right_shoulder = np.array(right_shoulder)  # Shoulder landmark
    right_elbow = np.array(right_elbow)  # Elbow landmark
    right_wrist = np.array(right_wrist)  # Wrist landmark
    
    # Calculate the angle between the vectors formed by the landmarks
    angle = np.degrees(np.arccos(np.dot(right_shoulder - right_elbow, right_wrist - right_elbow) / (np.linalg.norm(right_shoulder - right_elbow) * np.linalg.norm(right_wrist - right_elbow))))
    
    return angle


In [18]:
right_shoulder = [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y]
right_elbow = [landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].y]
right_wrist = [landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].y]


In [19]:
right_shoulder, right_elbow, right_wrist

([0.3199075162410736, 0.12152671068906784],
 [0.23667044937610626, 0.4232039153575897],
 [0.2163674384355545, 0.6843536496162415])

In [20]:
calculate_overhead_press_angle(right_shoulder, right_elbow, right_wrist)

169.0205573165704

In [21]:
tuple(np.multiply(right_shoulder, [640, 480]).astype(int))
tuple(np.multiply(right_elbow, [640, 480]).astype(int))
tuple(np.multiply(right_wrist, [640, 480]).astype(int))

(138, 328)

In [22]:
cap = cv2.VideoCapture(0)

## Setup mediapipe instance
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()
        
        # Recolor image to RGB
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
      
        # Make detection
        results = pose.process(image)
    
        # Recolor back to BGR
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        # Extract landmarks
        try:
            landmarks = results.pose_landmarks.landmark
            
            # Get coordinates for ohp landmarks
            right_shoulder = [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y]
            right_elbow = [landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].y]
            right_wrist = [landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].y]
            
            # Calculate ohp angle
            angle = calculate_overhead_press_angle(right_shoulder, right_elbow, right_wrist)
            
            # Visualize angle
            cv2.putText(image, str(angle), 
                           tuple(np.multiply(right_elbow, [640, 480]).astype(int)), 
                           cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA
                                )
                       
        except:
            pass
        
        
        # Render detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2), 
                                mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2) 
                                 )               
        
        cv2.imshow('Overhead Press Checker', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()


In [23]:
cap = cv2.VideoCapture(0)

# OHP counter variables
counter = 0 
stage = None

## Setup mediapipe instance
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()
        
        # Recolor image to RGB
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
      
        # Make detection
        results = pose.process(image)
    
        # Recolor back to BGR
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        # Extract landmarks
        try:
            landmarks = results.pose_landmarks.landmark
            
            # Get coordinates for ohp landmarks
            right_shoulder = [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y]
            right_elbow = [landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].y]
            right_wrist = [landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].y]
            
            # Calculate ohp angle
            angle = calculate_overhead_press_angle(right_shoulder, right_elbow, right_wrist)
            
            # Visualize angle
            cv2.putText(image, str(angle), 
                           tuple(np.multiply(right_elbow, [640, 480]).astype(int)), 
                           cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA)
            
            # Overhead Press Counter Algorithm
            if angle < 90:
                stage = "down"
            if angle > 150 and stage =="down":
                stage="up"
                counter +=1
                print(counter)
                       
        except:
            pass
        
        # Render Overhead Press Counter
        # Setup status box
        cv2.rectangle(image, (0,0), (225,73), (245,117,16), -1)
        
        # Rep data
        cv2.putText(image, 'BENCH PRESS', (15,12), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA)
        cv2.putText(image, str(counter), 
                    (10,60), 
                    cv2.FONT_HERSHEY_SIMPLEX, 2, (255,255,255), 2, cv2.LINE_AA)
        
        # Stage data
        cv2.putText(image, '   STAGE', (65,12), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA)
        cv2.putText(image, stage, 
                    (60,60), 
                    cv2.FONT_HERSHEY_SIMPLEX, 2, (255,255,255), 2, cv2.LINE_AA)
        
        # Render detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2), 
                                mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2) 
                                 )               
        
        cv2.imshow('Bench Press Tracker', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

    cap.release()
    
    cv2.destroyAllWindows()


1
2
3
4
5
6
7
8
9
